In [2]:
import os
import pandas as pd
import numpy as np
import json
import re
import nltk
import gensim

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora, models

from nltk.stem import WordNetLemmatizer

In [3]:
json_file = open('/Users/Nico/Desktop/DSDATA' + '/consolidated.json')
json_data = json.load(json_file)
document = pd.DataFrame(json_data)

In [5]:
document.head(40)

,source,date,title,article_body,author
0,https://edition.cnn.com/2021/01/31/football/ma...,2021-01-31,Manchester United star Marcus Rashford raciall...,"The 23-year-old, who recently received an MBE ...","By Ben Church, CNN"
1,https://edition.cnn.com/2021/01/31/europe/pola...,2021-01-31,'Women's rights are being trampled',Editor's note: The people photographed and in...,"By Kuba Kaminski, Antonia Mortensen and Laura ..."
2,https://edition.cnn.com/2021/01/31/uk/tom-moor...,2021-01-31,100-year-old UK fundraising hero Tom Moore hos...,Moore became a national celebrity and was kni...,"By Nic Robertson and Eoin McSweeney, CNN"
3,https://edition.cnn.com/2021/01/31/football/pa...,2021-01-31,Palmeiras fans gather to celebrate dramatic Co...,"Only 5,000 invited guests were permitted into ...","By Ben Church, CNN"
4,https://edition.cnn.com/2014/02/28/world/europ...,2021-01-31,Ukraine Fast Facts,"Only 5,000 invited guests were permitted into ...",cannot access
5,https://edition.cnn.com/2021/01/30/europe/fort...,2021-01-30,Austrian man leaves fortune to French village ...,"Erich Schwam, a Jewish refugee who arrived in...","By Eoin McSweeney and Arnaud Siad, CNN"
6,https://edition.cnn.com/2021/01/30/uk/uk-asylu...,2021-01-30,Five men arrested after asylum center set abla...,The fire broke out on Friday at the Napier Bar...,"By Arnaud Siad and Schams Elwazer, CNN"
7,https://edition.cnn.com/2021/01/30/football/ch...,2021-01-30,Chelsea left 'disgusted' by online racism dire...,"The 21-year-old said ""something needs to chang...","By Ben Church, CNN"
8,https://edition.cnn.com/2021/01/30/europe/russ...,2021-01-30,Russian activist Navalny's foundation calls on...,Navalny's Anti-Corruption Foundation (FBK) sai...,"By Zahra Ullah and Matthew Chance, CNN"
9,https://edition.cnn.com/2021/01/29/europe/eu-a...,2021-01-30,EU regulators recommend authorization of Astra...,The long-awaited decision from the European Me...,"By Laura Smith-Spark, Tara John and James Frat..."


In [6]:
def removeURLFromText(text):
    result = re.sub(r"http\S+", "", text)
    result = result.strip()
    return result

In [7]:
processed_docs = document['article_body'].map(removeURLFromText)
processed_docs.head()

0    The 23-year-old, who recently received an MBE ...
1    Editor's note: The people photographed and int...
2    Moore became a national celebrity and was knig...
3    Only 5,000 invited guests were permitted into ...
4    Only 5,000 invited guests were permitted into ...
Name: article_body, dtype: object

In [8]:
nltk.download('wordnet')

def lemmatize_stemming(text):
    return WordNetLemmatizer().lemmatize(text, pos = 'v')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Nico\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result


In [10]:
processed_docs = processed_docs.map(preprocess)
processed_docs.head()

0    [year, recently, receive, recognition, campaig...
1    [editor, note, people, photograph, interview, ...
2    [moore, national, celebrity, knight, queen, el...
3    [invite, guests, permit, janeiro, iconic, mara...
4    [invite, guests, permit, janeiro, iconic, mara...
Name: article_body, dtype: object

In [11]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [12]:
for x in range (0, 20):
    print(x, ":" ,dictionary[x])

0 : abhorrence
1 : abuse
2 : account
3 : action
4 : albion
5 : algorithms
6 : announce
7 : anthony
8 : appal
9 : appear
10 : arrest
11 : arsenal
12 : assess
13 : authorities
14 : available
15 : axel
16 : ban
17 : beautiful
18 : begin
19 : bias


In [13]:
dictionary.cfs

{161: 21,
 113: 4,
 111: 13,
 114: 2,
 25: 1,
 53: 2,
 155: 5,
 30: 6,
 92: 12,
 97: 6,
 128: 2,
 84: 5,
 152: 8,
 1: 10,
 132: 15,
 83: 18,
 94: 7,
 123: 6,
 20: 1,
 78: 12,
 102: 1,
 34: 11,
 60: 22,
 54: 6,
 42: 5,
 160: 4,
 65: 5,
 61: 1,
 48: 2,
 11: 1,
 134: 2,
 79: 4,
 139: 2,
 109: 2,
 131: 3,
 129: 6,
 126: 1,
 73: 1,
 68: 2,
 91: 3,
 17: 2,
 32: 2,
 56: 8,
 86: 11,
 110: 11,
 26: 5,
 33: 25,
 39: 9,
 98: 11,
 77: 15,
 95: 18,
 2: 2,
 28: 2,
 125: 112,
 47: 3,
 105: 10,
 43: 2,
 40: 2,
 115: 1,
 74: 4,
 59: 29,
 107: 1,
 148: 1,
 7: 2,
 80: 2,
 15: 2,
 151: 2,
 146: 9,
 90: 5,
 157: 15,
 87: 8,
 106: 4,
 143: 2,
 9: 11,
 93: 2,
 72: 9,
 96: 12,
 10: 2,
 141: 1,
 104: 1,
 158: 3,
 23: 1,
 4: 1,
 85: 3,
 121: 1,
 124: 1,
 57: 7,
 55: 9,
 44: 5,
 18: 5,
 24: 12,
 3: 5,
 49: 4,
 62: 3,
 21: 4,
 99: 4,
 138: 12,
 153: 12,
 0: 2,
 101: 1,
 37: 11,
 159: 12,
 13: 7,
 118: 6,
 58: 8,
 137: 6,
 147: 18,
 145: 9,
 75: 2,
 82: 10,
 127: 5,
 133: 9,
 31: 9,
 154: 3,
 35: 14,
 100: 6,
 66:

In [14]:
dictionary.dfs

{161: 13,
 113: 3,
 111: 6,
 114: 2,
 25: 1,
 53: 2,
 155: 2,
 30: 3,
 92: 6,
 97: 6,
 128: 2,
 84: 3,
 152: 5,
 1: 3,
 132: 8,
 83: 10,
 94: 2,
 123: 5,
 20: 1,
 78: 7,
 102: 1,
 34: 6,
 60: 7,
 54: 3,
 42: 5,
 160: 3,
 65: 2,
 61: 1,
 48: 2,
 11: 1,
 134: 2,
 79: 4,
 139: 2,
 109: 2,
 131: 2,
 129: 5,
 126: 1,
 73: 1,
 68: 2,
 91: 2,
 17: 1,
 32: 1,
 56: 7,
 86: 6,
 110: 7,
 26: 5,
 33: 11,
 39: 8,
 98: 6,
 77: 6,
 95: 6,
 2: 2,
 28: 2,
 125: 20,
 47: 3,
 105: 2,
 43: 2,
 40: 2,
 115: 1,
 74: 4,
 59: 8,
 107: 1,
 148: 1,
 7: 2,
 80: 2,
 15: 2,
 151: 2,
 146: 6,
 90: 2,
 157: 9,
 87: 6,
 106: 2,
 143: 2,
 9: 5,
 93: 2,
 72: 2,
 96: 5,
 10: 2,
 141: 1,
 104: 1,
 158: 2,
 23: 1,
 4: 1,
 85: 2,
 121: 1,
 124: 1,
 57: 3,
 55: 5,
 44: 2,
 18: 5,
 24: 9,
 3: 3,
 49: 3,
 62: 3,
 21: 4,
 99: 4,
 138: 7,
 153: 8,
 0: 2,
 101: 1,
 37: 6,
 159: 9,
 13: 7,
 118: 3,
 58: 5,
 137: 4,
 147: 10,
 145: 9,
 75: 2,
 82: 7,
 127: 3,
 133: 6,
 31: 6,
 154: 3,
 35: 4,
 100: 3,
 66: 2,
 112: 2,
 135: 1,
 71

In [15]:
dictionary.num_pos

6054

In [16]:
dictionary.num_docs

50

In [17]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [18]:
bow_corpus

[[(0, 1),
  (1, 6),
  (2, 1),
  (3, 2),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 2),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 2),
  (23, 1),
  (24, 2),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 2),
  (31, 1),
  (32, 2),
  (33, 1),
  (34, 3),
  (35, 2),
  (36, 1),
  (37, 2),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 2),
  (45, 2),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 4),
  (52, 1),
  (53, 1),
  (54, 1),
  (55, 3),
  (56, 1),
  (57, 3),
  (58, 1),
  (59, 3),
  (60, 2),
  (61, 1),
  (62, 1),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 1),
  (67, 1),
  (68, 1),
  (69, 1),
  (70, 1),
  (71, 4),
  (72, 4),
  (73, 1),
  (74, 1),
  (75, 1),
  (76, 1),
  (77, 5),
  (78, 1),
  (79, 1),
  (80, 1),
  (81, 2),
  (82, 1),
  (83, 4),
  (84, 2),
  (85, 1),
  (86, 1),
  (87, 1),
  (88, 1),
  (89, 1),
  (90, 2),
  (91, 1)

In [19]:
tfidf = models.TfidfModel(bow_corpus)

In [20]:
corpus_tfidf = tfidf[bow_corpus]
for x in corpus_tfidf:
    print(x)

[(0, 0.05447055556542688), (1, 0.2856550916161916), (2, 0.05447055556542688), (3, 0.09521836387206388), (4, 0.06620015125905866), (5, 0.05447055556542688), (6, 0.038964873476345224), (7, 0.05447055556542688), (8, 0.05447055556542688), (9, 0.038964873476345224), (10, 0.05447055556542688), (11, 0.06620015125905866), (12, 0.04760918193603194), (13, 0.033271013054795816), (14, 0.04760918193603194), (15, 0.05447055556542688), (16, 0.05447055556542688), (17, 0.13240030251811732), (18, 0.038964873476345224), (19, 0.05447055556542688), (20, 0.06620015125905866), (21, 0.04274095987179512), (22, 0.07175917248480032), (23, 0.06620015125905866), (24, 0.05803642522601042), (25, 0.06620015125905866), (26, 0.038964873476345224), (27, 0.04274095987179512), (28, 0.05447055556542688), (29, 0.033271013054795816), (30, 0.09521836387206388), (31, 0.03587958624240016), (32, 0.13240030251811732), (33, 0.025622417042677013), (34, 0.10763875872720048), (35, 0.08548191974359023), (36, 0.04274095987179512), (37,

In [21]:
document.dtypes

source          object
date            object
title           object
article_body    object
author          object
dtype: object

In [22]:
processednews = []

source = document['source']
date = document['date']
title = document['title']
article_body = document['article_body']
author = document['author']

for i in range (0, len(corpus_tfidf)):
    processednews.append({
        'source' : source[i],
        'date' : date[i],
        'title' : title[i],
        'author' : author[i],
        'article_body_bow' : corpus_tfidf[i]
    })
processednews

[{'source': 'https://edition.cnn.com/2021/01/31/football/marcus-rashford-manchester-united-racism-spt-intl/index.html',
  'date': '2021-01-31',
  'title': 'Manchester United star Marcus Rashford racially abused online',
  'author': 'By Ben Church, CNN',
  'article_body_bow': [(0, 0.05447055556542688),
   (1, 0.2856550916161916),
   (2, 0.05447055556542688),
   (3, 0.09521836387206388),
   (4, 0.06620015125905866),
   (5, 0.05447055556542688),
   (6, 0.038964873476345224),
   (7, 0.05447055556542688),
   (8, 0.05447055556542688),
   (9, 0.038964873476345224),
   (10, 0.05447055556542688),
   (11, 0.06620015125905866),
   (12, 0.04760918193603194),
   (13, 0.033271013054795816),
   (14, 0.04760918193603194),
   (15, 0.05447055556542688),
   (16, 0.05447055556542688),
   (17, 0.13240030251811732),
   (18, 0.038964873476345224),
   (19, 0.05447055556542688),
   (20, 0.06620015125905866),
   (21, 0.04274095987179512),
   (22, 0.07175917248480032),
   (23, 0.06620015125905866),
   (24, 0.058

,source,date,title,article_body,author,article_body_bow
0,https://edition.cnn.com/2021/01/31/football/ma...,2021-01-31,Manchester United star Marcus Rashford raciall...,"The 23-year-old, who recently received an MBE ...","By Ben Church, CNN","(0, 0.057612261110489386)"
1,https://edition.cnn.com/2021/01/31/europe/pola...,2021-01-31,'Women's rights are being trampled',Editor's note: The people photographed and in...,"By Kuba Kaminski, Antonia Mortensen and Laura ...","(3, 0.019701355531614395)"
2,https://edition.cnn.com/2021/01/31/uk/tom-moor...,2021-01-31,100-year-old UK fundraising hero Tom Moore hos...,Moore became a national celebrity and was kni...,"By Nic Robertson and Eoin McSweeney, CNN","(22, 0.03988006710861786)"
3,https://edition.cnn.com/2021/01/31/football/pa...,2021-01-31,Palmeiras fans gather to celebrate dramatic Co...,"Only 5,000 invited guests were permitted into ...","By Ben Church, CNN","(13, 0.05078496476865932)"
4,https://edition.cnn.com/2014/02/28/world/europ...,2021-01-31,Ukraine Fast Facts,"Only 5,000 invited guests were permitted into ...",cannot access,"(13, 0.05078496476865932)"
5,https://edition.cnn.com/2021/01/30/europe/fort...,2021-01-30,Austrian man leaves fortune to French village ...,"Erich Schwam, a Jewish refugee who arrived in...","By Eoin McSweeney and Arnaud Siad, CNN","(30, 0.11646966595769927)"
6,https://edition.cnn.com/2021/01/30/uk/uk-asylu...,2021-01-30,Five men arrested after asylum center set abla...,The fire broke out on Friday at the Napier Bar...,"By Arnaud Siad and Schams Elwazer, CNN","(8, 0.05429478241239185)"
7,https://edition.cnn.com/2021/01/30/football/ch...,2021-01-30,Chelsea left 'disgusted' by online racism dire...,"The 21-year-old said ""something needs to chang...","By Ben Church, CNN","(0, 0.0685521608594647)"
8,https://edition.cnn.com/2021/01/30/europe/russ...,2021-01-30,Russian activist Navalny's foundation calls on...,Navalny's Anti-Corruption Foundation (FBK) sai...,"By Zahra Ullah and Matthew Chance, CNN","(10, 0.0299815402468912)"
9,https://edition.cnn.com/2021/01/29/europe/eu-a...,2021-01-30,EU regulators recommend authorization of Astra...,The long-awaited decision from the European Me...,"By Laura Smith-Spark, Tara John and James Frat...","(1, 0.0205731307457109)"


0    (0, 0.057612261110489386)
1    (3, 0.019701355531614395)
2    (22, 0.03988006710861786)
3    (13, 0.05078496476865932)
4    (13, 0.05078496476865932)
Name: article_body_bow, dtype: object